In [1]:
!zipline bundles

Korea_commodity 2023-12-05 21:01:47.126702
Korea_commodity 2023-12-05 21:01:08.694331
Korea_commodity 2023-12-05 09:01:34.202227
csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>


In [2]:
!zipline ingest -b Korea_commodity

[2023-12-06T12:13:35+0600-INFO][zipline.data.bundles.core]
 Ingesting Korea_commodity
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]   


In [11]:
from zipline.api import order_target_percent, symbol, set_commission, set_benchmark, order,record, order_target

In [3]:
def EMA(day, price_hist):
    #price_hist = price_hist.tolist()
    multiplier = 2 / (day + 1)
    EMA_list = []
    EMA_prev = price_hist[0]
    EMA_list.append(EMA_prev)
    for price in price_hist[1:]:
        EMA = (price - EMA_prev) * multiplier + EMA_prev
        EMA_list.append(EMA)
        EMA_prev = EMA
    return EMA_list

In [24]:
def initialize(context):
    context.i = 0
    context.asset = symbol("Kor_gold_mod")


def handle_data(context, data):
    context.i += 1
    if context.i < 30:  # Ensure there are enough days for both short and long moving averages
        return

    # Calculate short-term (fast) and long-term (slow) moving averages
    short_mavg = data.history(context.asset, 'close', bar_count=15, frequency='1d').mean()
    long_mavg = data.history(context.asset, 'close', bar_count=30, frequency='1d').mean()

    # Trading logic
    if short_mavg > long_mavg:
        # Buy signal
        order_target(context.asset, 1)
    elif short_mavg < long_mavg:
        # Sell signal
        order_target(context.asset, 0)

    # Record values for analysis
    record(
        Kor_gold=data.current(context.asset, 'close'),
        short_mavg=short_mavg,
        long_mavg=long_mavg
    )


In [25]:
import pandas as pd
from zipline.utils.calendar_utils import get_calendar
from zipline import run_algorithm
calendar = get_calendar('XKRX')

In [26]:

start_date = pd.to_datetime('2022-11-22 00:00:00.000000', format='%Y-%m-%d %H:%M:%S.%f')
end_date = pd.to_datetime('2023-05-02 00:00:00.000000', format='%Y-%m-%d %H:%M:%S.%f')
results = run_algorithm(start=start_date,
                            end=end_date,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=10000,
                            benchmark_returns=None,
                            trading_calendar=calendar,
                            bundle='Korea_commodity',
                            data_frequency='daily',
                            )

/home/popeye/.local/lib/python3.10/site-packages/zipline/finance/ledger.py:424: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self.daily_returns_series[session_ix] = self.todays_returns
/home/popeye/.local/lib/python3.10/site-packages/empyrical/stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
/home/popeye/.local/lib/python3.10/site-packages/empyrical/stats.py:799: RuntimeWarning: invalid value encountered in divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


In [27]:
results

,period_open,period_close,gross_leverage,net_leverage,short_exposure,long_exposure,starting_exposure,transactions,positions,pnl,capital_used,orders,short_value,long_value,shorts_count,longs_count,returns,portfolio_value,ending_cash,starting_cash,ending_value,starting_value,ending_exposure,benchmark_period_return,benchmark_volatility,alpha,beta,sharpe,sortino,max_drawdown,algo_volatility,max_leverage,excess_return,algorithm_period_return,treasury_period_return,trading_days,period_label,Kor_gold,short_mavg,long_mavg
2022-11-22 06:30:00+00:00,2022-11-22 00:01:00+00:00,2022-11-22 06:30:00+00:00,0.000000,0.000000,0.0,0.0,0.0,[],[],0.0,0.0,[],0.0,0.0,0,0,0.000000,10000.000,10000.000,10000.000,0.0,0.0,0.0,0.0,NaN,None,None,NaN,NaN,0.000000,NaN,0.000000,0.0,0.000000,0.0,1,2022-11,NaN,NaN,NaN
2022-11-23 06:30:00+00:00,2022-11-23 00:01:00+00:00,2022-11-23 06:30:00+00:00,0.000000,0.000000,0.0,0.0,0.0,[],[],0.0,0.0,[],0.0,0.0,0,0,0.000000,10000.000,10000.000,10000.000,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.0,2,2022-11,NaN,NaN,NaN
2022-11-24 06:30:00+00:00,2022-11-24 00:01:00+00:00,2022-11-24 06:30:00+00:00,0.000000,0.000000,0.0,0.0,0.0,[],[],0.0,0.0,[],0.0,0.0,0,0,0.000000,10000.000,10000.000,10000.000,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.0,3,2022-11,NaN,NaN,NaN
2022-11-25 06:30:00+00:00,2022-11-25 00:01:00+00:00,2022-11-25 06:30:00+00:00,0.000000,0.000000,0.0,0.0,0.0,[],[],0.0,0.0,[],0.0,0.0,0,0,0.000000,10000.000,10000.000,10000.000,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.0,4,2022-11,NaN,NaN,NaN
2022-11-28 06:30:00+00:00,2022-11-28 00:01:00+00:00,2022-11-28 06:30:00+00:00,0.000000,0.000000,0.0,0.0,0.0,[],[],0.0,0.0,[],0.0,0.0,0,0,0.000000,10000.000,10000.000,10000.000,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.0,5,2022-11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-25 06:30:00+00:00,2023-04-25 00:01:00+00:00,2023-04-25 06:30:00+00:00,4.625742,4.625742,0.0,85000.0,84600.0,[],"[{'sid': Equity(0 [KOR_GOLD_MOD]), 'amount': 1...",400.0,0.0,[],0.0,85000.0,0,1,0.022253,18375.432,-66624.568,-66624.568,85000.0,84600.0,85000.0,0.0,0.0,None,None,2.656605,4.796505,-0.143484,0.608113,8.055951,0.0,0.837543,0.0,107,2023-04,85000.0,84759.333333,83156.333333
2023-04-26 06:30:00+00:00,2023-04-26 00:01:00+00:00,2023-04-26 06:30:00+00:00,4.529698,4.529698,0.0,85500.0,85000.0,[],"[{'sid': Equity(0 [KOR_GOLD_MOD]), 'amount': 1...",500.0,0.0,[],0.0,85500.0,0,1,0.027210,18875.432,-66624.568,-66624.568,85500.0,85000.0,85500.0,0.0,0.0,None,None,2.745510,4.963631,-0.143484,0.606098,8.055951,0.0,0.887543,0.0,108,2023-04,85500.0,84814.000000,83373.000000
2023-04-27 06:30:00+00:00,2023-04-27 00:01:00+00:00,2023-04-27 06:30:00+00:00,4.511096,4.511096,0.0,85600.0,85500.0,[],"[{'sid': Equity(0 [KOR_GOLD_MOD]), 'amount': 1...",100.0,0.0,[],0.0,85600.0,0,1,0.005298,18975.432,-66624.568,-66624.568,85600.0,85500.0,85600.0,0.0,0.0,None,None,2.753292,4.977514,-0.143484,0.603289,8.055951,0.0,0.897543,0.0,109,2023-04,85600.0,84843.333333,83549.666667
2023-04-28 06:30:00+00:00,2023-04-28 00:01:00+00:00,2023-04-28 06:30:00+00:00,4.586704,4.586704,0.0,85200.0,85600.0,[],"[{'sid': Equity(0 [KOR_GOLD_MOD]), 'amount': 1...",-400.0,0.0,[],0.0,85200.0,0,1,-0.021080,18575.432,-66624.568,-66624.568,85200.0,85600.0,85200.0,0.0,0.0,None,None,2.653999,4.787429,-0.143484,0.601974,8.055951,0.0,0.857543,0.0,110,2023-04,85200.0,84802.000000,83713.000000
